<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Преобразование-матрицы-признаков" data-toc-modified-id="Преобразование-матрицы-признаков-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Преобразование матрицы признаков</a></span></li><li><span><a href="#Проверка-алгоритма-преобразования" data-toc-modified-id="Проверка-алгоритма-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка алгоритма преобразования</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ниму было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [5]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

data = pd.read_csv('/datasets/insurance.csv')

Посмотрим датасет

In [6]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Разделим выборку на признаки и таргет

In [7]:
X = data.drop('Страховые выплаты', axis=1)
y = data['Страховые выплаты']

Переведем их в массивы

In [8]:
X = np.array(X)
y = np.array(y)

## Преобразование матрицы признаков

Попробуем умножить наши признаки на обратимую матрицу и посмотрим, как будут соотноситься вектора предсказаний моделей a и a' в этом случае 

Дано:

$$
a = Xw
$$


$X$ — матрица признаков

$w$ — вектор весов линейной регрессии


$$
a' = X'w'
$$


$X' = XP$ — матрица признаков, умноженная на обратимую матрицу


$w'$ — измененный вектор весов линейной регрессии



Рассмотрим w':


$$
w' = ((XP)^T (XP))^{-1}(XP)^T y =
$$


Откроем скобки транспонирования:


$$
(P^T X^T XP)^{-1}P^T X^T y = 
$$


Откроем скобку обращения:


$$
(X^T XP)^{-1}(P^T)^{-1} P^T X^T y = 
$$


$(P^T)^{-1} P^T $ - единичная матрица, сократим ее и откроем скобки обращения:


$$
P^{-1}(X^T X)^{-1}X^T y
$$


Подставим полученное выражение в $a'=X'w'$:


$$
a' = XP P^{-1}(X^T X)^{-1}X^T y
$$


$P P^{-1}$ - единичная матрица, сократим ее:


$$
a' = X(X^T X)^{-1}X^T y
$$


Найдем выражение $a = Xw$:


$$
a = X(X^T X)^{-1}X^T y
$$


Как мы видим, оно равно a', следовательно:


$$
a' = a
$$



Вывод: при умножении матрицы признаков на обратимую матрицу, вектора предсказаний не изменяются

## Проверка алгоритма преобразования

Зададим квадратную матрицу со стороной, равной количеству наших признаков

In [9]:
P = np.random.rand(X.shape[1], X.shape[1])
print(P)

[[0.83709962 0.66204819 0.3303428  0.65178409]
 [0.38407241 0.65073294 0.85864506 0.93456412]
 [0.88607221 0.70356763 0.43140177 0.65780323]
 [0.44001459 0.64548601 0.63927332 0.12264014]]


Проверим ее обратимость

In [10]:
print(inv(P))

[[-11.47872641  -1.08381712  13.1965424   -1.51810332]
 [ 19.87582621   0.13755863 -20.52204771   3.39338056]
 [-12.64731712   0.40637918  12.06367912  -0.58703826]
 [  2.49781018   1.04627904  -2.21758353  -1.19955963]]


Получим измененную матрицу признаков

In [11]:
XP = X.dot(P)

Сделаем класс линейной регрессии, который будет прибавлять нулевой признак, считать веса и возвращать метрику r2

In [12]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
    def r2_score(self, target, predictions):
        return r2_score(target, predictions)


Сделаем предсказания на обоих моделей

In [13]:
model = LinearRegression()
model.fit(X, y)
predict = model.predict(X)
print(predict)

[ 0.51172715  0.68431581  0.09373365 ... -0.25697046 -0.19099161
  0.04904991]


In [ ]:
model_P = LinearRegression()
model_P.fit(XP, y)
predict_P = model_P.predict(XP)
print(predict_P)

Найдем самую большую разницу между предсказаниями

In [ ]:
print(max(abs(predict - predict_P)))

Сравним модели по метрике r2

In [ ]:
r2 = model.r2_score(y, predict)

In [ ]:
r2_P = model_P.r2_score(y, predict_P)

Найдем разницу

In [ ]:
print(r2 - r2_P)

Вывод: умножение матрицы признаков на обратимую практически не сказывается на качестве работы модели

## Общий вывод

Мы предположили алгоритм шифрования данных - умножение матрицы признаков на обратимую матрицу. Мы показали математически, что предсказания моделей на исходной и полученной матрицах признаков должны быть равны. Далее, мы построили модели линейных регрессий и сравнили полученные предсказания и метрику r2. Результаты подтвердили теоретические утверждения:

- самая большая разница в признаках равна 7.5e-06
- разница в метрике r2 равна 1.3e-11

Т.е разница в качестве полученных моделей близка к нулевой